In [116]:
import os
os.chdir(r"C:/Users/Admin/Desktop/Kristal/advisory_vols_gen")
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [117]:
# Reading assets_vol file 
assets_vol = pd.read_csv('assets_vol.csv')
assets_vol['VOL'] = assets_vol['VOL']/100
assets_vol = assets_vol.astype({'asset_id' : int})

In [118]:
# Creating separate df for multiple assets & single assets
mult_assets = assets_vol.loc[(assets_vol['kristal_type'] == 'multiple_assets')].drop(['kristal_type'], axis=1)
mult_assets.drop(mult_assets[(mult_assets.asset_id == 508)].index, inplace=True)
mult_assets['PROXY_INDEX'] = mult_assets['PROXY_INDEX'].str.upper() 

# Portfolio vol = asset vol for single-asset kristals
single_asset = assets_vol.loc[(assets_vol['kristal_type'] == 'single_asset')]
single_asset['portfolio_vol'] = single_asset['VOL']

In [119]:
# Function for calculating correlation part of the portfolio vol
import itertools
assets_corr = pd.read_excel("assets_bbg.xlsx", sheet_name="Correlation")
assets_corr = assets_corr.set_index('Tickers')
assets_corr.columns = map(str.upper, assets_corr.columns)
assets_corr.index = assets_corr.index.str.upper()

def calc_corr(bbg_df):
    asset_pairs = list(itertools.combinations(bbg_df['PROXY_INDEX'], 2))
    corr_vol = 0
    for pair in asset_pairs:
        asset1 = pair[0]
        asset2 = pair[1]
        correlation = assets_corr.loc[[asset1], [asset2]].values[0][0]
        w1 = bbg_df.loc[bbg_df.PROXY_INDEX == asset1]['asset_weight'].values[0]
        w2 = bbg_df.loc[bbg_df.PROXY_INDEX == asset2]['asset_weight'].values[0]
        v1 = bbg_df.loc[bbg_df.PROXY_INDEX == asset1]['VOL'].values[0]
        v2 = bbg_df.loc[bbg_df.PROXY_INDEX == asset2]['VOL'].values[0]
        corr_vol += w1*w2*correlation*v1*v2
    return corr_vol

In [120]:
# Creating lists of assets and corresponding values to get 1 row for each kristal
mult_assets = mult_assets.groupby(['kristal_name'], as_index=False).agg(lambda x: x.tolist())

In [121]:
# Generating individual vols sum
mult_assets['vol_sum'] = mult_assets.apply(lambda x : sum([a*b for a,b in zip(x.asset_weight, x.VOL)]), axis=1)

# Generating correlation part 
mult_assets['vol_corr'] = mult_assets.apply(lambda x : calc_corr(pd.DataFrame(list(zip(x.PROXY_INDEX, x.asset_weight, x.VOL)), columns=['PROXY_INDEX', 'asset_weight', 'VOL'])), axis=1)

# Portfolio vol = sqrt of parts from above 2 values
mult_assets['portfolio_vol'] = mult_assets.apply(lambda x : (((x.vol_sum ** 2) + 2*x.vol_corr) ** 0.5), axis=1)

In [122]:
sa_portfolio_vol = single_asset[['kristal_name', 'portfolio_vol']]
ma_portfolio_vol = mult_assets[['kristal_name', 'portfolio_vol']]
portfolio_vol = sa_portfolio_vol.append([ma_portfolio_vol], ignore_index=True)

In [123]:
portfolio_vol.to_csv('portfolio_vol.csv', index=False)